In [1]:
import pandas as pd
import numpy as np
import re, os
from os.path import join
from tqdm import tqdm
from file_function import read_filelist
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl import Workbook
import warnings
warnings.simplefilter("ignore")

# dtype 정의
debt_dtype = {'채무자키':str, '타채무자키':str, '담당자키':str, "변경전":str, "담당변경사유":str}


def 키워드로파일명찾기_last(폴더:str, 포함키워드:str, 제외키워드="", 전체경로=True) :
    file_list = [file for file in os.listdir(폴더) if not file.startswith('~$')]
    if 제외키워드 == "" : 
        fn = [file for file in file_list if re.search(포함키워드, file)]
    else : 
        fn = [file for file in file_list if (re.search(포함키워드, file)!=None) & (re.search(제외키워드,file)==None)]
    
    if len(fn) >= 1 :
        print("읽을 파일 : ", fn[-1])
        if 전체경로 : 
            return join(폴더,fn[-1])
        else : 
            return fn[-1]
    else : 
        print(f"포함키워드:{포함키워드}, 제외키워드{제외키워드} 조건을 만족하는 파일이 없습니다.")

### 담당자 변경
- 배정 : 관리팀 > 추심팀 
- 제외 : 추심팀 > 관리팀

##### 기본변수 및 담당자로그파일 읽기

In [2]:
# 추심팀 명단 : 부서 == 추심 
# 사원 다운로드 : 직위는 팀관리까지 / 사원성명,지점,부서만 체크해서 다운 OR 기본값이나 전체도 상관은 없음

# 주간 담당자로그 파일 읽기
######################################################################
wd = r"D:\3.자산\기타 업무\담당자변경"
기간 = "241112~241118"  # 전주 화요일 ~ 이번주 월요일
# filename_log = "담당자로그_20241008_1029.xlsx"
# 대성filename_log = "대성_담당자로그_20241008_1029.xlsx"
# 사원 = "사원_20241008_1029.xlsx"
######################################################################
# 사원파일 읽기
전사원명단 = pd.read_excel(키워드로파일명찾기_last(wd, "사원"), dtype=str).fillna("")
    # 추심팀, 필요칼럼만 : 성명, 지점만
사원_지점 = 전사원명단.query('부서=="추심"')[["사원성명", "지점"]].copy() # 사원키로 작업하는게 나으려나?


    # 대성 추가 #####################################################
사원_지점.loc[len(사원_지점)] = ['권영진', 'JM신용정보']
사원_지점.loc[len(사원_지점)] = ['박민재', 'JM신용정보']
사원_지점.loc[len(사원_지점)] = ['박용상', 'JM신용정보']
사원_지점.loc[len(사원_지점)] = ['조준일', 'JM신용정보']
####################################################################

사원_지점.reset_index(drop=True, inplace=True)

    # 추심팀 이름만  리스트로
추심팀 = 사원_지점.사원성명.tolist()
print(len(추심팀), 추심팀)

# 담당자 변경 로그 읽기
log = pd.read_excel(키워드로파일명찾기_last(wd, "담당자", "대성"), dtype=debt_dtype).fillna("")
대성log = pd.read_excel(키워드로파일명찾기_last(wd, "대성_담당자"), dtype=debt_dtype).fillna("")
    # 통합
log = pd.concat([log, 대성log], axis=0)

    # 채무자키로 그룹화 : 변경일순으로 정렬되어 있으므로 순회하면 된다.
log_grouped = log.groupby('채무자키')
log_grouped.get_group(list(log_grouped.groups.keys())[0])

읽을 파일 :  사원_20241105_1114.xlsx
39 ['김대홍', '김효중', '도현관', '신경용', '임헌규', '한웅희', '권영진', '박민재', '박용상', '신용규', '조준일', '김순희', '김진욱', '김찬영', '김형철', '김홍민', '김홍일', '김희수', '박경순', '변상영', '변정섭', '소범모', '송경숙', '원종영', '육군택', '이명섭', '이문정', '이병곤', '이승우', '이영자', '이은경', '이은미', '최금자', '최진철', '한명철', '권영진', '박민재', '박용상', '조준일']
읽을 파일 :  담당자로그_20241119_0907.xlsx
읽을 파일 :  대성_담당자로그_20241119_0906.xlsx


,채무자키,채무자,변경전,변경후,담당변경사유,요청자,변경일,변경자
19,10830631,고수빈,종결요청,종결(조정),개인회생 면책종결,오재홍,2024-11-15 09:11:29,오재홍


##### 데이터 조작

In [3]:
# 바꾸면 안 되는 담당자 변경건 있는지
log_copy = log.copy()
전 = log_copy.변경전
후 = log_copy.변경후
# 종결건 바꾸는 경우
cond1 = (전=="종결(매각)") | (전=="DS")
# 신규조정진입 후 기존조정 기각,폐지,실효
cond2 = ((전.str.contains("신용")) & (후.str.contains("폐지|기각"))) | ((전.str.contains("개인")) & (후.str.contains("실효|기각"))) | ((전.str.contains("파산")) & (후.str.contains("폐지|실효")))

log_copy["변경실수체크"] = np.where(cond1 | cond2, "체크", "")

체크 = log_copy.query("변경실수체크=='체크'").sort_values('채무자키')
if len(체크) > 0 :
    print(len(체크),'건 체크!! ----------------------------------------')
    체크.to_excel(join(wd, 기간+"변경실수인제체크할것.xlsx"), index=False)


# 제외건, 배정건 만들기

# 제외건
제외_채무자키_list = []
제외_채무자명_list = []
제외_변경전_list = []
제외_변경후_list = []
제외_최종변경일_list = []
제외_변경사유_list = []

# 배정건
배정_채무자키_list = []
배정_채무자명_list = []
배정_변경전_list = []
배정_변경후_list = []
배정_최종변경일_list = []
배정_변경사유_list = []


for k, v in log_grouped :
    변경전추심팀 = list(x for x in v.변경전.values if x in 추심팀)
    변경전관리팀 = list(x for x in v.변경전.values if x not in 추심팀)
    최종담당자 = v.변경후.values[-1]

    # 제외건에 해당하는 조건 : 변경전에 추심팀 이름이 있고(빈리스트가 아닌 경우), 최종담당자가 추심팀이 아닌 경우
    if 변경전추심팀 and  최종담당자 not in 추심팀 : 
        제외_채무자키_list.append(k) # k는 str임
        제외_채무자명_list.append(v.채무자.values[0]) # 똑같으므로
        제외_변경전_list.append(변경전추심팀[-1]) # 여러명인 경우, 마지막 담당자
        제외_변경후_list.append(최종담당자)
        제외_최종변경일_list.append(v.변경일.values[-1][:10])
        제외_변경사유_list.append(v.담당변경사유.values[-1])
    
    # 배정건에 해당하는 조건
    if 변경전관리팀 and  최종담당자 in 추심팀 : 
        배정_채무자키_list.append(k) # k는 str임
        배정_채무자명_list.append(v.채무자.values[0]) # 똑같으므로
        배정_변경전_list.append(">".join(변경전관리팀)) # 진행상황 알수 있도록 > 로 연결
        배정_변경후_list.append(최종담당자)
        배정_최종변경일_list.append(v.변경일.values[-1][:10])
        배정_변경사유_list.append(v.담당변경사유.values[-1])

# df만들기
columns = ["채무자키", "채무자명", "변경전", "변경후", "최종변경일", "변경사유"]
제외_data = {col: values for col, values in zip(columns, [제외_채무자키_list, 제외_채무자명_list, 제외_변경전_list, 제외_변경후_list, 제외_최종변경일_list,제외_변경사유_list])}
제외건 = pd.DataFrame(제외_data)

배정_data = {col: values for col, values in zip(columns, [배정_채무자키_list, 배정_채무자명_list, 배정_변경전_list, 배정_변경후_list, 배정_최종변경일_list,배정_변경사유_list])}
배정건 = pd.DataFrame(배정_data)

# 지점열 추가
제외건 = pd.merge(제외건,사원_지점, left_on='변경전', right_on='사원성명', how='left')
제외건.drop('사원성명', axis=1, inplace=True)

배정건 = pd.merge(배정건,사원_지점, left_on='변경후', right_on='사원성명', how='left')
배정건.drop('사원성명', axis=1, inplace=True)

# 지점-담당자 이름순 정렬
제외건.sort_values(['지점', '변경전', '변경후'], ascending=True, inplace=True)
배정건.sort_values(['지점', '변경후', '변경전'], ascending=True, inplace=True)

print("제외건 전체 : ", len(제외건))
print("배정건 전체 : ", len(배정건))




# 엑셀만들기 --------------------------------------------------------------------------------------------------------------
# 솔림(not 새한), 새한 구분
솔림_제외건 = 제외건.query('지점=="광주" or 지점=="서울"')
솔림_배정건 = 배정건.query('지점=="광주" or 지점=="서울"')

JM_제외건 = 제외건.query('지점=="JM신용정보"')
JM_배정건 = 배정건.query('지점=="JM신용정보"')

OK_제외건 = 제외건.query('지점=="OK신용정보"')
OK_배정건 = 배정건.query('지점=="OK신용정보"')


# 엑셀 쓰기

# 제외건, 배정건 받아서 각각을 시트로 하는 wb 반환하는 함수
def make_excel_file(제외건, 배정건) : 

    # 시트 만들기
    wb = Workbook()
    ws1 = wb.active
    ws1.title = "제외건"
    ws2 = wb.create_sheet("배정건")


    # 시트에 데이터 입력
    for r in dataframe_to_rows(제외건, index=False, header=True) : ws1.append(r)
    for r in dataframe_to_rows(배정건, index=False, header=True) : ws2.append(r)

    # 스타일 지정
    # 폰트
    font_data = Font(name='NN30', size=10)
    font_col = Font(name='NN30', size=10, color='FFFFFF', bold=True)
    # 컬럼 배경색
    fill_col = PatternFill(fill_type='solid', start_color='FF0072C6', end_color='FF0072C6') # 칼럼명
    fill_highlight = PatternFill(fill_type='solid', start_color='F79646', end_color='F79646') # 칼럼명
    # 테두리 : 직선
    border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
    # 정렬
    alignment = Alignment(horizontal='center', vertical='center')

    # 스타일 적용
    for ws in [ws1, ws2] : 

        # 시트 첫행(칼럼명)
        for cell in ws[1] :
            if (cell.value =="변경전" and ws.title=="제외건") or (cell.value =="변경후" and ws.title=="배정건") :
                cell.fill = fill_highlight

            else :     
                cell.fill = fill_col
            cell.font = font_col
            cell.border = border
            cell.alignment = alignment
        
        # 2행부터 순회(데이터)
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row) : 
            for cell in row :
                cell.font = font_data
                cell.border = border
                cell.alignment = alignment
    
    return wb
# 파일 저장
print(f'솔림 제외 : {len(솔림_제외건)}, 배정 : {len(솔림_배정건)}')
print(f'JM 제외 : {len(JM_제외건)}, 배정 : {len(JM_배정건)}')
print(f'OK 제외 : {len(OK_제외건)}, 배정 : {len(OK_배정건)}')
make_excel_file(솔림_제외건, 솔림_배정건).save(join(wd, "result", "솔림_담당자변경_"+기간+".xlsx"))
make_excel_file(JM_제외건, JM_배정건).save(join(wd, "result", "JM_담당자변경_"+기간+".xlsx"))
make_excel_file(OK_제외건, OK_배정건).save(join(wd, "result", "OK_담당자변경_"+기간+".xlsx"))

제외건 전체 :  61
배정건 전체 :  4
솔림 제외 : 27, 배정 : 2
JM 제외 : 32, 배정 : 2
OK 제외 : 2, 배정 : 0


##### 결과 확인